# 获取城市名单

In [1]:
import requests
import pyquery
import re
import pandas as pd
import random

In [2]:
user_agent = 'Mozilla/5.0 (compatible; Baiduspider/2.0; +http://www.baidu.com/search/spider.html)'
url = 'http://data.acmr.com.cn/member/city/city_md.asp'
headers = {'User-Agent':user_agent}
r1 = requests.get(url = url,headers = headers)

In [3]:
r1.encoding = 'gbk'
pq = pyquery.PyQuery(r1.text)
maintext_tag = pq('body > font > table > tr > td > table > tr > td > table > tr > td > table > tr > td > table > tr > td > table.maintext') 

In [4]:
city_super = maintext_tag.eq(0).text()
city_super = city_super.strip(r'[TOP]').split(':')[1].split(' ')[1:-1]

In [5]:
city_metro = maintext_tag.eq(1).text()
city_metro = city_metro.strip(r'[TOP]').split(':')[1].split(' ')[1:-1]

In [6]:
city_big = maintext_tag.eq(2).text()
city_big = city_big.strip(r'[TOP]').split(':')[1].split(' ')[1:-1]

In [7]:
city_middle = maintext_tag.eq(3).text()
city_middle = city_middle.strip(r'[TOP]').split(':')[1].split(' ')[1:-1]

In [8]:
city_small = maintext_tag.eq(4).text()
city_small = city_small.strip(r'[TOP]').split(':')

In [9]:
city_small_list = city_small[1].strip(r'县级市').split(' ')[1:-1]
city_small_list.extend(city_small[2].split(' ')[1:-1])

In [10]:
population = ['超大','特大','大','中等','小']
city_list = [city_super,city_metro,city_big,city_middle,city_small_list]
with open('city_info.csv',mode = 'w',encoding = 'utf-8') as city_info:
    city_info.write('city,population\n')
    
    for i in range(len(city_list)):
        cities = city_list[i]
        for city in cities:
            city_info.write('{},{}\n'.format(city,population[i]))

# 数据清洗

In [11]:
city_df = pd.read_csv('city_info.csv')

In [12]:
city_df['province'] = city_df.city.str.extract(r'(.+?)省',expand=False)

province_other = ['北京市','天津市','上海市','重庆市','内蒙','新疆','广西','宁夏','西藏']
for i in province_other:
    city_df.loc[city_df.city.str.contains(i),'province'] = i

In [26]:
city_df['name'] = city_df['city'].str.extract(r'(.+?)市',expand=False)
city_df['Name'] = city_df['name']
city_df.loc[city_df['Name'].str.contains('省'),'name'] = city_df['name'].str.extract(r'省(.+)',expand=False)   
for i in province_other:
    city_df.loc[city_df.name.str.contains(i),'name'] = city_df.name.str.replace(i,'') 


city_df = city_df[['city','population','province','name']]
city_df.index.name = 'id'

In [27]:
city_df['name_first'] = city_df.name.str.extract(r'(^.{1})',expand=False)
city_df['name_last'] = city_df.name.str.extract(r'(.{1}$)',expand=False)

In [15]:
def chinese_to_py(a):
    a = str(a.encode('unicode_escape'))[-5:-1].upper()
    b = dic_py[a]
    return b            

In [16]:
dic_py = {}
with open('unicode_py.txt','r') as f:
    for i in f.readlines():
        dic_py[i.split()[0]] = i.split()[1]

In [28]:
name_first_py_list = []
for a in city_df['name_first']:
    name_first_py_list.append(chinese_to_py(a))
    
city_df['name_first_py'] = pd.Series(name_first_py_list)

In [29]:
name_last_py_list = []
for a in city_df['name_last']:
    name_last_py_list.append(chinese_to_py(a))
    
city_df['name_last_py'] = pd.Series(name_last_py_list)

# 游戏

In [19]:
def get_basic(name_city):
    return city_df.loc[city_df['name'] == name_city,['population','province','name']] 

In [20]:
def get_info(name_city):
    url_city = 'https://baike.baidu.com/item/{}'.format(name_city)
    r2 = requests.get(url = url_city,headers = headers)
    r2.encoding = 'utf-8'
    pq2 = pyquery.PyQuery(r2.text)
    return name_city,pq2('body > div > div > div > div > div > div').text().split(name_city,1)[1] 

In [22]:
def show(val):
    color = 'green'
    return 'color:s%'% color

In [30]:
name_city = input('请输入开始城市：')
def game_continue(name_city):
    name_last_py = city_df.loc[city_df['name'] == name_city,'name_last_py']
    name_last_py = name_last_py.values[0][:-1]

    name_last = city_df.loc[city_df['name_first_py'].str.contains(name_last_py),'name'].values
    print(name_last)
    if name_last.any(0):
        name_city = random.choice(list(name_last))
        print('接龙城市:{}'.format(name_city))
        user_input = input('请按以下提示继续:\nc：继续游戏\nt：停止游戏\nb：城市基本信息\ni：详细信息\n')    
        if user_input == 'c':
            name_city = name_city
            game_continue(name_city)
        elif user_input == 't':
            print('游戏停止')
            return None
        elif user_input == 'b':
            print(get_basic(name_city))
            game_continue(name_city)
        elif user_input == 'i':
            print(get_info(name_city))
            game_continue(name_city)
        
        else:
            print('输入不正确，游戏结束')
            return None
    else:
        print('无可接龙的城市，本次游戏结束')
        return None

game_continue(name_city)

请输入开始城市：北京
['荆州' '景德镇' '荆门' '靖江' '井冈山' '景洪']
接龙城市:荆门
请按以下提示继续:
c：继续游戏
t：停止游戏
b：城市基本信息
i：详细信息
b
    population province name
id                          
145         中等       湖北   荆门
['孟州']
接龙城市:孟州
请按以下提示继续:
c：继续游戏
t：停止游戏
b：城市基本信息
i：详细信息
t
游戏停止
